In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
import seaborn as sns
import math
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [ ]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # Path relative to your mahcine I guess ? 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test) # ????
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
def partition_data(x, y, i):
    "Merge data and labels"
    data = np.c_[y, x]
    
    "Find indexes where the 23rd element of the data is 'i' (i = 0, 1, 2 and 3)"
    index = data[:, 23]==i
    
    "Split the data according to the indexes found"
    data_i = data[index, :]
    "Unmerge the data and labels"
    y_i = data_i[:, 0]
    x_i = data_i[:, 1:]
    
    "-999s don't appear in 1st column following the indexes, so we don't take it in consideration in the following"
    "and we get rid of the then 21st column containing the 'i's"
    x_test = np.delete(x_i[:, 1:], 21, axis=1)
    
    "Remove columns where elements have value -999"
   
    x_test = np.delete(x_test, np.where(x_test[0]==-999), axis=1)
    
    "Concactenation of 1st column of the data and the rest with elements of value -999 removed"
    x_test = np.c_[x_i[:, 0], x_test]
    
    "Replace elements of value -999 in 1st column of the data by the mean of "
    "Here, we can do a weighted mean instead of simply taking the mean "
    for column in range(x_test[0].size):
        indices = x_test[:,column]==-999
        x_test[indices, column] = np.mean(x_test[~indices, column])

    return index, y_i, x_test

In [ ]:
index0, y0, x_test0 = partition_data(tX, y, 0)
index1, y1, x_test1 = partition_data(tX, y, 1)
index2, y2, x_test2 = partition_data(tX, y, 2)
index3, y3, x_test3 = partition_data(tX, y, 3)

In [ ]:
def compute_corr(x_test):
    corr_mat = np.corrcoef(x_test,rowvar=False)
    return np.where(np.abs(corr_mat)>=0.95, 1,0)

def visualize_corr(corr):
    fig, ax = plt.subplots(figsize=(corr.shape[0],corr.shape[1]))
    sns.heatmap(corr, vmax=1.0, center=0, fmt='.2f',
            square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70})
    plt.show();

def standardize_data(x):
    """Standardize the original data set."""
    mean_x = np.mean(x)
    x = x - mean_x
    std_x = np.std(x)
    x = x / std_x
    return x

We standardize the 4 batches of data.

In [ ]:

std_test0 = standardize_data(x_test0)
std_test1 = standardize_data(x_test1)
std_test2 = standardize_data(x_test2)
std_test3 = standardize_data(x_test3)


We try to find any correlations between features of the standardized data, by taking a look at their correlation matrices. 

In [ ]:

corr_mat_0 = compute_corr(std_test0[:,:-1])
corr_mat_1 = compute_corr(std_test1) 
corr_mat_2 = compute_corr(std_test2)
corr_mat_3 = compute_corr(std_test3)
    
corrs = [corr_mat_0,corr_mat_1,corr_mat_2,corr_mat_3]

for elem in corrs:
    visualize_corr(elem)

We observe that among the 4 batches of partitioned data, the first batch has a feature which is strictly correlated to the other ! 
Furthermore, for the second, thrid , and fourth batch, we observe that some features here and there have very high correlation coefficients, greater than 0.95 ... 

In [ ]:
def run_model(y,tx,initial_w,max_iters=10000,gamma=0.05,model=least_squares_GD,lambda_=0):
   
    loss, w = model(y,tx,initial_w,max_iters,gamma) #Change for future algorithms 
   
    return w
    
    

In [ ]:
def try_T(data,labels,prop,gamma):
    train_prop = int(data.shape[0]*prop)
    
    train = data[:train_prop] 
    y_train = labels[:train_prop]
    y_test = labels[train_prop:]
    test  = data[train_prop:]
    
    rand_w = [np.random.uniform(-1,1) for x in range(data.shape[1])]
    
    w = run_model(y_train,train,rand_w,gamma = gamma)
    return np.mean(predict_labels(w,test)==y_test)

In [ ]:
try_T(std_test0,y0,0.9,0.05)
      

In [ ]:
def cross_validation(K,y_batch,tx_batch,max_iters,gamma,model):
   
    N = tx_batch.shape[0]
    fold_size  = tx_batch.shape[0]//K #euclidean divison => cast as int
    
    accuracies = [] # nb of accuracies to compute = K 
    
    ws = []
    for i in range(0,tx_batch.shape[0]-1,fold_size):
        
        #Define the indexes of the training set
        start = i
        end = i + fold_size
        
        #Define training set
        train_tx = np.vstack((tx_batch[:start],tx_batch[end:N-1]))
        print(train_tx.shape)
        train_y = np.array(list(y_batch[:start])+list(y_batch[end:N-1]))
        print(train_y.shape)
        
        # for each run of the model start with a random w ( Is this a good idea ? )
        rand_initial = [np.random.uniform(-10,10) for x in range(tx_batch.shape[1])]
        w= run_model(train_y,train_tx,rand_initial,max_iters,gamma,model) #rewrite .... 
        
        # Test w
        ws.append(w)
        acc = np.mean(predict_labels(w,tx_batch[start:end])==y[start:end])
        accuracies.append(acc)
        
        print(acc)
        
    print("This is ws \n")
    print(ws[-1])
    return np.mean(accuracies)
        
        
        
        
        
    
    

In [ ]:
cross_validation(11,y0,std_test0,max_iters=1000,gamma=0.0000001,model=least_squares_GD)